In [ ]:
# Step 1: Change HycomHindcastArchive settings
# downloadOptions.maxTimesPerFile from 24 to 8
# subsetOptions.timeStride from 1 to 3

# Step 2: trigger stageFiles to put the new files in the database
#hindcast_archive_to_change.stageFiles('HycomHindcastDataSource', hindcast_archive_to_change.subsetOptions, hindcast_archive_to_change.downloadOptions)

# Step 3: Trigger download
# job = c3.HycomHindcastDownloadJob(
#         **{
#             'options': c3.HycomHindcastDownloadJobOptions().toJson()
#         }
#     ).upsert()
# job.start()

# Step 4: remove old files from the databse
# see Andy code

In [1]:
from ocean_navigation_simulator.utils.misc import get_c3
c3 = get_c3()

Starting to connect to c3


## 1. Change HycomHindcastArchive settings

In [4]:
# Dict of ids for each region that is not GOM
hc_archive_ids = {
1: "6af1e19b-3aa1-480c-8433-83ab3a0fc2f3",
2: "ffc90ea7-bac3-45dd-aefc-b4b3ee1ed40d",
3: "15b30061-0338-4ca4-bbe2-1ee1ce12222f",
4: "bbeadd84-1f55-4951-95d1-3c11dcd85ee5",
5: "6874690e-87a2-4c21-8ab0-dd95ada4a741",
6: "36319544-8f10-423c-8185-c8a808056681",
7: "6f600e6e-26ee-426a-a49f-2d8dec13ed8f"}
hc_archive_id = hc_archive_ids[1]

In [5]:
hindcast_archive_to_change = c3.HycomHindcastArchive.get(hc_archive_id)

In [7]:
# Analyze different region settings
print(hindcast_archive_to_change.downloadOptions.maxTimesPerFile, hindcast_archive_to_change.subsetOptions.timeStride)

24 3


In [ ]:
hindcast_archive_to_change.downloadOptions.maxTimesPerFile = 8
 # if we set it to 3, we get every third 3h forecast -> every 9th hour
hindcast_archive_to_change.subsetOptions.timeStride = 1

## 2: trigger stageFiles to put the new files in the database


In [ ]:
hindcast_archive_to_change.stageFiles('HycomHindcastDataSource', hindcast_archive_to_change.subsetOptions, hindcast_archive_to_change.downloadOptions)

In [ ]:
all_files = hindcast_archive_to_change.get('files')
print(len(all_files.files))

In [ ]:
not_downloaded = 0
for file in all_files.files:
    if file.get('status').status == 'not_downloaded':
        not_downloaded +=1
        print(file.id)
        # file.remove()

## 3. Trigger download

In [ ]:
job = c3.HycomHindcastDownloadJob(
        **{
            'options': c3.HycomHindcastDownloadJobOptions().toJson()
        }
    ).upsert()
job.start()

In [ ]:
c3.HycomHindcastDownloadJob.get(job.id).status()

## 4. Remove old files from the databse

In [ ]:
hindcast_archive_to_change = c3.HycomHindcastArchive.get(hc_archive_id)# Region 6
all_files = hindcast_archive_to_change.get('files')
print(len(all_files.files))

In [ ]:
# Analyze files that don't end the correct way
# Delete the ones you don't like
cnt_not_21 = 0
for file in all_files.files:
    if "T21:00:00" not in file.id:
        cnt_not_21 += 1
        print(file.id)
        # file.remove()
print(cnt_not_21)

In [ ]:
# Delete files that don't end the correct way